In [1]:
%pip install seleniumbase
%pip install bs4
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
from seleniumbase import Driver
from bs4 import BeautifulSoup
import os
import json
import re
import tqdm

In [ ]:
url = "https://arxiv.org/year/math-ph/2022"  # Put your yearly URL here

In [24]:
browsers = ["chrome", "firefox", "edge", "ie", "opera", "safari"]
for browser in browsers:
    try:
        driver = Driver(
            browser=browser,
            uc=True,
            headless2=True,
            incognito=True,
            agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36 AVG/112.0.21002.139",
            do_not_track=True,
            undetectable=True,
        )
        break
    except Exception as e:
        print(f"Error: {e}")
        continue

Error: Chrome not found! Install it first!


*** geckodriver to download = v0.34.0

https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-win64.zip ...
Download Complete!

Extracting ['geckodriver.exe'] from geckodriver-v0.34.0-win64.zip ...
Unzip Complete!

The file [geckodriver.exe] was saved to:
C:\Users\khoch\AppData\Local\Programs\Python\Python312\Lib\site-packages\seleniumbase\drivers\geckodriver.exe

Making [geckodriver.exe v0.34.0] executable ...
[geckodriver.exe v0.34.0] is now ready for use!

Error: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line



In [25]:
PATH = "https://arxiv.org/"  # Don't change

In [26]:
def run(URL, p="results"):
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    tag = soup.find("div", class_="morefewer")
    tag = tag.find_all("a")[-1]["href"][1:]
    driver.get(PATH + tag)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    t = dict()
    r = []
    for ele in soup.find_all(["dt", "dd"]):
        if ele.name == "dt":
            a = ele.find_all("a")
            pdf = os.path.join(PATH, a[1]["href"][1:])
            t["pdf"] = pdf
        elif ele.name == "dd":
            authors = ele.find("div", class_="list-authors").find_all("a")
            title = ele.find("div", class_="list-title mathjax").text.strip()
            match = re.search(r"Title:\s+(.*)", title)
            if match:
                title = match.group(1)
            t["title"] = title
            t["authors"] = [x.text for x in authors]
            r.append(t)
            t = dict()
    os.makedirs(p, exist_ok=True)
    nm = "-".join(URL.split("/")[-2:])
    with open(f"{p}/result_{nm}.json", "w") as f:
        json.dump(r, f, indent=4)

In [27]:
parts = url.split("/")
if parts[-1].isdigit() and len(parts[-1]) == 4:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    content_div = soup.find("div", id="content")
    li = [
        os.path.join(PATH, x.find("a")["href"][1:])
        for x in content_div.find("ul").find_all("li")
    ]
    for l in tqdm.tqdm(li, "Processing"):
        run(l)
else:
    run(url)

Processing:  50%|█████     | 6/12 [00:08<00:07,  1.28s/it]